https://www.kaggle.com/code/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm/script

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

In [4]:
NFOLDS = 3
SEED = 0
NROWS = None

In [6]:
data = pd.read_csv('data/application_train.csv')
test = pd.read_csv('data/application_test.csv')
prev = pd.read_csv('data/previous_application.csv')

In [7]:
categorical_feats = [f for f in data.columns if data[f].dtype=='object']

In [10]:
for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

In [11]:
gc.enable()

In [12]:
y_train = data['TARGET']
del data['TARGET']

In [14]:
prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype == 'object'
]

for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])
    
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

In [15]:
x_train = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
x_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

ntrain = x_train.shape[0]
ntest = x_test.shape[0]

excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

x_train = x_train[features]
x_test = x_test[features]

In [17]:
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

In [18]:
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]

class CatboostWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]
        
class LightGBMWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['feature_fraction_seed'] = seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]


class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))


def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)


et_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'nrounds': 200
}

catboost_params = {
    'iterations': 200,
    'learning_rate': 0.5,
    'depth': 3,
    'l2_leaf_reg': 40,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.7,
    'scale_pos_weight': 5,
    'eval_metric': 'AUC',
    'od_type': 'Iter',
    'allow_writing_files': False
}

lightgbm_params = {
    'n_estimators':200,
    'learning_rate':0.1,
    'num_leaves':123,
    'colsample_bytree':0.8,
    'subsample':0.9,
    'max_depth':15,
    'reg_alpha':0.1,
    'reg_lambda':0.1,
    'min_split_gain':0.01,
    'min_child_weight':2    
}

In [19]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf= CatBoostClassifier, seed = SEED, params=catboost_params)
lg = LightGBMWrapper(clf = LGBMClassifier, seed = SEED, params = lightgbm_params)

xg_oof_train, xg_oof_test = get_oof(xg)
et_oof_train, et_oof_test = get_oof(et)
rf_oof_train, rf_oof_test = get_oof(rf)
cb_oof_train, cb_oof_test = get_oof(cb)

print("XG-CV: {}".format(sqrt(mean_squared_error(y_train, xg_oof_train))))
print("ET-CV: {}".format(sqrt(mean_squared_error(y_train, et_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, rf_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, cb_oof_train))))

x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis=1)

print("{},{}".format(x_train.shape, x_test.shape))

logistic_regression = LogisticRegression()
logistic_regression.fit(x_train,y_train)

test['TARGET'] = logistic_regression.predict_proba(x_test)[:,1]

test[['SK_ID_CURR', 'TARGET']].to_csv('first_submission.csv', index=False, float_format='%.8f')

0:	total: 232ms	remaining: 46.3s
1:	total: 292ms	remaining: 28.9s
2:	total: 346ms	remaining: 22.7s
3:	total: 409ms	remaining: 20s
4:	total: 486ms	remaining: 19s
5:	total: 557ms	remaining: 18s
6:	total: 625ms	remaining: 17.2s
7:	total: 695ms	remaining: 16.7s
8:	total: 770ms	remaining: 16.3s
9:	total: 861ms	remaining: 16.4s
10:	total: 951ms	remaining: 16.3s
11:	total: 1.05s	remaining: 16.5s
12:	total: 1.16s	remaining: 16.7s
13:	total: 1.23s	remaining: 16.4s
14:	total: 1.32s	remaining: 16.3s
15:	total: 1.4s	remaining: 16.1s
16:	total: 1.47s	remaining: 15.8s
17:	total: 1.54s	remaining: 15.6s
18:	total: 1.62s	remaining: 15.4s
19:	total: 1.69s	remaining: 15.3s
20:	total: 1.77s	remaining: 15.1s
21:	total: 1.84s	remaining: 14.9s
22:	total: 1.93s	remaining: 14.9s
23:	total: 2.02s	remaining: 14.8s
24:	total: 2.1s	remaining: 14.7s
25:	total: 2.19s	remaining: 14.6s
26:	total: 2.26s	remaining: 14.5s
27:	total: 2.33s	remaining: 14.3s
28:	total: 2.41s	remaining: 14.2s
29:	total: 2.48s	remaining: 14s


42:	total: 3.44s	remaining: 12.6s
43:	total: 3.54s	remaining: 12.5s
44:	total: 3.63s	remaining: 12.5s
45:	total: 3.73s	remaining: 12.5s
46:	total: 3.81s	remaining: 12.4s
47:	total: 3.88s	remaining: 12.3s
48:	total: 3.97s	remaining: 12.2s
49:	total: 4.05s	remaining: 12.1s
50:	total: 4.12s	remaining: 12s
51:	total: 4.2s	remaining: 12s
52:	total: 4.28s	remaining: 11.9s
53:	total: 4.36s	remaining: 11.8s
54:	total: 4.46s	remaining: 11.8s
55:	total: 4.57s	remaining: 11.7s
56:	total: 4.66s	remaining: 11.7s
57:	total: 4.75s	remaining: 11.6s
58:	total: 4.85s	remaining: 11.6s
59:	total: 4.93s	remaining: 11.5s
60:	total: 5.01s	remaining: 11.4s
61:	total: 5.08s	remaining: 11.3s
62:	total: 5.15s	remaining: 11.2s
63:	total: 5.22s	remaining: 11.1s
64:	total: 5.3s	remaining: 11s
65:	total: 5.36s	remaining: 10.9s
66:	total: 5.44s	remaining: 10.8s
67:	total: 5.52s	remaining: 10.7s
68:	total: 5.59s	remaining: 10.6s
69:	total: 5.67s	remaining: 10.5s
70:	total: 5.74s	remaining: 10.4s
71:	total: 5.83s	remai

84:	total: 6.76s	remaining: 9.14s
85:	total: 6.83s	remaining: 9.06s
86:	total: 6.9s	remaining: 8.97s
87:	total: 6.98s	remaining: 8.89s
88:	total: 7.06s	remaining: 8.81s
89:	total: 7.13s	remaining: 8.72s
90:	total: 7.21s	remaining: 8.63s
91:	total: 7.29s	remaining: 8.56s
92:	total: 7.38s	remaining: 8.49s
93:	total: 7.46s	remaining: 8.41s
94:	total: 7.54s	remaining: 8.34s
95:	total: 7.63s	remaining: 8.26s
96:	total: 7.7s	remaining: 8.18s
97:	total: 7.78s	remaining: 8.1s
98:	total: 7.86s	remaining: 8.01s
99:	total: 7.93s	remaining: 7.93s
100:	total: 8.02s	remaining: 7.86s
101:	total: 8.1s	remaining: 7.78s
102:	total: 8.19s	remaining: 7.71s
103:	total: 8.27s	remaining: 7.63s
104:	total: 8.36s	remaining: 7.56s
105:	total: 8.44s	remaining: 7.48s
106:	total: 8.52s	remaining: 7.4s
107:	total: 8.62s	remaining: 7.34s
108:	total: 8.71s	remaining: 7.27s
109:	total: 8.79s	remaining: 7.19s
110:	total: 8.88s	remaining: 7.12s
111:	total: 8.96s	remaining: 7.04s
112:	total: 9.04s	remaining: 6.96s
113:	t